# SimRank
----
Similarity measure from the **automorphic** equivalence family. However, the SimRank family unlike the RoleSim, do not satisfy all the axioms for the automorphic equivalence, but can be used as a measure for quantifying it.

In [1]:
import pandas as pd
import numpy as np

from SimRank import SimRank

In [2]:
def get_n_pairs(matrix, sim_score=0.8):
    '''
    Helper function to return the number of similar pairs
    with the desired similarity score.
    '''
    return np.count_nonzero(~np.isnan(matrix[matrix >= sim_score])) - matrix.shape[0]

## Postings and Replies

In [2]:
postings_replies = pd.read_parquet('data/df_edge_list_directed_users_postings_replies.parquet')

In [3]:
postings_replies.head()

,ID_CommunityIdentity_Source,ID_CommunityIdentity_Target,count_posting_replies
0,30,57450,1
1,30,99495,1
2,30,117560,1
3,30,125092,1
4,30,133175,1


In [4]:
len(postings_replies)

1247835

In [5]:
len(np.unique(postings_replies['ID_CommunityIdentity_Source']))

11292

The dataset contains 1.2 million edges and 11.2 thousand nodes.

### Random sampling

Tried random sampling, to subsample the graph which had imapct on the convergence rate of the algorithm. The convergence was slower as the sample got smaller:
- with 5% of the data, algorithm converged after 6 iterations
- with 10% of the data, algorithm converged after 5 iterations
- with 20% of the data, algorithm converged after 5 iterations
- with 50% of the data, algorithm converged after 4 iterations
- with all data, algorithm converged after 4 iterations.

It also influences the number of similar pairs found (given the similarity score of 0.8 or bigger), and the number of pairs decreases as the sample size increases. So with a larger graph we get fewer and fewer pairs which indicates that this algorithm is not very well suite for our task overall. Even with the smallest sample size, we get a very small detection of similarity and the similarity score values are low, we would like to see 0.9 scores ideally. 

In [ ]:
import time
sample_sizes = [0.05, 0.1, 0.2, 0.5]

for s in sample_sizes:
    start_time = time.time()
    sr = SimRank.SimRank()
    result = sr.fit(
        postings_replies.sample(frac=s, random_state=123), 
        weighted = True, 
        from_node_column = 'ID_CommunityIdentity_Source', 
        to_node_column = 'ID_CommunityIdentity_Target',
        weight_column = 'count_posting_replies')
    n_pairs = get_n_pairs(result)
    print(f"Size {s} took {(time.time() - start_time) / 60} minutes and found {n_pairs} similar node pairs.")

Start iterating...
Percent: [##############################] 100% Complete! 
Converged at iteration 6Size 0.05 took 8.340412676334381 minutes and found 1100 similar node pairs.
Start iterating...
Percent: [##############################] 100% Complete! 
Converged at iteration 5Size 0.1 took 9.864646609624227 minutes and found 748 similar node pairs.
Start iterating...
Percent: [##############################] 100% Complete! 
Converged at iteration 5Size 0.2 took 12.72212127049764 minutes and found 482 similar node pairs.
Start iterating...
Percent: [##############################] 100% Complete! 
Converged at iteration 4Size 0.5 took 12.333981005350749 minutes and found 280 similar node pairs.


### Weighted
----
In this version of the algorithm, the weights of the edges are taken into account.

In [6]:
sr = SimRank.SimRank()
sr_postings_replies = sr.fit(
    postings_replies, 
    weighted = True, 
    from_node_column = 'ID_CommunityIdentity_Source', 
    to_node_column = 'ID_CommunityIdentity_Target',
    weight_column = 'count_posting_replies')

Start iterating...
Percent: [##############################] 100% Complete! 
Converged at iteration 4

In [10]:
sr_postings_replies.shape

(11643, 11643)

In [16]:
get_n_pairs(sr_postings_replies)

106

Applying the SimRank to the weighted graph (no subsampling) produced only 106 node pairs with a similarity score larger than 0.8 (excluding the pairs when each node is paired with itself), which is still pretty low as we would be ideally looking for values around 0.9 which we do not obtain.  Therefore, this is a very poor performance as this is a very small portion of all the pairs in the resulting matrix.

### Unweighted
----
Computation on the full graph was computationally infeasible, thus we use the subsampling with 50% of the data.

In [7]:
sr = SimRank.SimRank()
sr_postings_replies_unweighted = sr.fit(
    postings_replies.sample(frac=0.5, random_state=123), 
    weighted = False, 
    from_node_column = 'ID_CommunityIdentity_Source', 
    to_node_column = 'ID_CommunityIdentity_Target')

Start iterating...
Percent: [##############################] 100% Complete! 
Converged at iteration 7

In [8]:
sr_postings_replies_unweighted

,589825,589828,4,524311,524312,196633,30,589854,196641,524324,...,589748,589751,589756,524227,589777,65490,524248,65505,524279,131065
589825,1.000000,0.000896,0.001052,0.000930,0.000933,0.000909,0.000862,0.000850,0.000876,0.000963,...,0.001830,0.000934,0.000911,0.000898,0.000922,0.000901,0.0,0.000892,0.000910,0.000827
589828,0.000896,1.000000,0.000894,0.000885,0.000899,0.000891,0.000869,0.000878,0.000870,0.000897,...,0.000884,0.000879,0.000903,0.000874,0.000927,0.000889,0.0,0.000878,0.000887,0.000872
4,0.001052,0.000894,1.000000,0.000929,0.000919,0.000991,0.000863,0.000928,0.000930,0.000947,...,0.000944,0.000909,0.000916,0.000796,0.000924,0.000896,0.0,0.000938,0.000922,0.000806
524311,0.000930,0.000885,0.000929,1.000000,0.000889,0.000899,0.001167,0.000888,0.000958,0.000906,...,0.001184,0.001087,0.001375,0.000886,0.001113,0.001043,0.0,0.001149,0.001028,0.000920
524312,0.000933,0.000899,0.000919,0.000889,1.000000,0.000898,0.000870,0.000871,0.004873,0.000898,...,0.001234,0.000886,0.001517,0.000878,0.000896,0.000881,0.0,0.000881,0.001819,0.000901
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65490,0.000901,0.000889,0.000896,0.001043,0.000881,0.000888,0.000870,0.002500,0.001886,0.000896,...,0.000975,0.001650,0.001304,0.000873,0.001253,1.000000,0.0,0.000883,0.001004,0.000929
524248,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000
65505,0.000892,0.000878,0.000938,0.001149,0.000881,0.000875,0.000867,0.001986,0.001210,0.000898,...,0.001130,0.001059,0.001392,0.000848,0.001188,0.000883,0.0,1.000000,0.001047,0.000930
524279,0.000910,0.000887,0.000922,0.001028,0.001819,0.000893,0.001334,0.001828,0.000868,0.000918,...,0.001251,0.001181,0.001229,0.000879,0.001227,0.001004,0.0,0.001047,1.000000,0.000902


In [19]:
sr_postings_replies_unweighted.columns = sr_postings_replies_unweighted.columns.astype(str)
sr_postings_replies_unweighted.to_parquet('output/sr_postings_replies_unweighted.parquet')

In [11]:
get_n_pairs(sr_postings_replies_unweighted)

296

Again, the amount of identified similar pairs is very low.

### Find the most similar node for each node

In [16]:
np.fill_diagonal(sr_postings_replies_unweighted.values, 0)

In [15]:
sr_postings_replies_unweighted.idxmax()

589825     31975
589828    575780
4          58571
524311    322433
524312     79981
           ...  
65490     208548
524248    589825
65505     248509
524279    580202
131065    502653
Length: 11344, dtype: int64

We could instead aim to capture the most similar node for each node, eventhough the scores are very low, likely not representing much of a similarity at all.

## Votes to Postings

In [3]:
votes_postings = pd.read_parquet('data/df_edge_list_directed_users_votes_to_postings_net.parquet')

In [21]:
votes_postings.head()

,ID_CommunityIdentity_Source,ID_CommunityIdentity_Target,count_votes_to_postings_net
0,4,6293,1
1,4,8008,1
2,4,9525,1
3,4,18764,1
4,4,22470,1


In [22]:
len(votes_postings)

6105250

In [23]:
len(np.unique(votes_postings['ID_CommunityIdentity_Source']))

12351

In [5]:
sr = SimRank.SimRank()
sr_votes_postings = sr.fit(
    votes_postings.sample(frac=0.5, random_state=123), 
    weighted = True, 
    from_node_column = 'ID_CommunityIdentity_Source', 
    to_node_column = 'ID_CommunityIdentity_Target',
    weight_column = 'count_votes_to_postings_net')

Start iterating...
Percent: [##############################] 100% Complete! 
Converged at iteration 4

In [7]:
sr_votes_postings.columns = sr_votes_postings.columns.astype(str)
sr_votes_postings.to_parquet('output/sr_votes_postings.parquet')

In [10]:
get_n_pairs(sr_votes_postings)

16

In [4]:
sr = SimRank.SimRank()
sr_votes_postings_unweighted = sr.fit(
    votes_postings.sample(frac=0.2, random_state=123), 
    weighted = False, 
    from_node_column = 'ID_CommunityIdentity_Source', 
    to_node_column = 'ID_CommunityIdentity_Target')

Start iterating...
Percent: [##############################] 100% Complete! 
Converged at iteration 5

In [5]:
sr_votes_postings_unweighted.columns = sr_votes_postings_unweighted.columns.astype(str)
sr_votes_postings_unweighted.to_parquet('output/sr_votes_postings_unweighted.parquet')

In [6]:
get_n_pairs(sr_votes_postings_unweighted)

56

Since the graph capturing the votes is three times larger than the reviews one, we opt to subsample 50% of the edges for the weighted case, and 20% for the unweighted case to be able to compute at least a portion of the similarity matrix. Again, the results are very unsatisfactory and even worse than in the previous graph.